In [ ]:
#https://peerj.com/articles/453/
from skimage import data, io
import skimage.filters as filters
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

image = cv.imread('/home/rcardiff/PNGImages/neg_CL1.png',0)
image_crop = image[500:1000, 0:120]
edges = filters.sobel(image_crop)
io.imshow(edges)


In [ ]:
fig, ax = plt.subplots(1, 1)
ax.hist(image.ravel(), bins=8, range=[0, 256]) #ravel returns flattened 1d array
ax.set_xlim(0, 256);

In [ ]:
def image_show(image, nrows=1, ncols=1, cmap='gray'):
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(14, 14))
    ax.imshow(image, cmap='gray')
    ax.axis('off')
    return fig, ax

In [ ]:
from skimage.filters import threshold_adaptive

bw = threshold_adaptive(image, 95, offset=-10)
image_show(bw)


In [ ]:
from skimage import feature

edges = feature.canny(image_crop, sigma=1,
                     low_threshold=1,
                     high_threshold=25)

image_show(edges)

In [ ]:
image_threshold = filters.threshold_mean(image) 
image_show(image > image_threshold);

In [ ]:
# Label image regions.
from skimage.measure import regionprops
import matplotlib.patches as mpatches
from skimage.morphology import label

label_image = label(edges)
print(len(label_image))
print(len(label_image[0]))
print(label_image[200])

#ax5 = plt.axes()
fig, ax5 = plt.subplots(figsize=(14, 14))
ax5.imshow(image, cmap=plt.cm.gray)
ax5.axis('off')

print(type(regionprops(label_image)))
print(len(regionprops(label_image)))
for region in regionprops(label_image):
    # Draw rectangle around segmented coins.
    minr, minc, maxr, maxc = region.bbox
    rect = mpatches.Rectangle((minc, minr),
                              maxc - minc,
                              maxr - minr,
                              fill=False,
                              edgecolor='red',
                              linewidth=1)
    ax5.add_patch(rect)

plt.tight_layout()
plt.show()

In [ ]:
props = regionprops(label_image)
props[10].area

In [ ]:
from skimage.morphology import watershed
from scipy import ndimage as ndi

from skimage.morphology import watershed
from skimage.feature import peak_local_max

distance = ndi.distance_transform_edt(image)
local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((3, 3)),
                            labels=image)
markers = ndi.label(local_maxi)[0]
labels = watershed(-distance, markers, mask=image)

fig, axes = plt.subplots(ncols=3, figsize=(10, 10), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(image, cmap=plt.cm.gray)
ax[0].set_title('Overlapping objects')
ax[1].imshow(-distance, cmap=plt.cm.gray)
ax[1].set_title('Distances')
ax[2].imshow(labels, cmap=plt.cm.nipy_spectral)
ax[2].set_title('Separated objects')



In [ ]:
print(len(labels))